# DROPPING DATAPOINTS

import packages : pandas

input file : final_cleaned_data.csv

stage 0 : typeconversion (all datatypes are either String or Int64)

stage 1 : drop all properties without prices or priced smaller than 25.000 €

stage 2 : drop all subtypes that are not houses or apartments

stage 3 : keep only properties with living area between 10 m² and 40.000 m²

stage 4 : drop all properties with terraces over 150m²

output file: filtered_final_cleaned_data.csv

In [1]:
import pandas as pd

#load input file into dataframe
df = pd.read_csv("final_cleaned_data.csv")
total_before = df.shape[0]

def convert_datatypes(df):

    # converting float to int
    df = df.apply(lambda x: x.astype("Int64") if x.dtype == float and (x.dropna() % 1 == 0).all() else x)

    # converting objects to strings
    df['property_ID'] = df['property_ID'].astype('string')
    df['locality_name'] = df['locality_name'].astype('string')
    df['type'] = df['type'].astype('string')
    df['subtype'] = df['subtype'].astype('string')
    df['state_of_building'] = df['state_of_building'].astype('string')
    return df

def drop_prices(df):
    #check number of rows in initial list
    before_drop = df.shape[0]

    # removing properties that do not have the price and are priced below 25K
    df = df.dropna(subset=["price (€)"])
    df = df[(df['price (€)'] >= 25000)]
    print ("dropped", before_drop - df.shape[0], "properties with a price less than 25K or no price") 

    return df

def drop_subtypes (df):
    #check number of rows in initial list
    before_drop = df.shape[0]
    #print(df.shape[0])
    #make a list of what subtypes we want to keep
    allowed = [
        "Apartment", "Residence", "Villa", "Ground floor", "Penthouse",
        "Mixed building", "Duplex", "Studio", "Chalet", "Master house",
        "Bungalow", "Cottage", "Loft", "Triplex", "Mansion"]

    # filter the subtypes we want to keep
    df = df[df['subtype'].isin(allowed)]

    #how many dropped?
    print("dropped",before_drop-df.shape[0],"properties with wrong subtypes")
    return (df)

def drop_living_area(df):
    #check number of rows in initial list
    before_drop = df.shape[0]

    # filter df: remove all houses under 10m² and above 400000
    df = df[(df['living_area (m²)'] >= 10) & (df['living_area (m²)'] < 40000)]

    #how many dropped?
    print("dropped",before_drop-df.shape[0],"properties with abnormal living areas")
    return(df)

def drop_terrace_area(df):
    #check number of rows in initial list
    before_drop = df.shape[0]

    # filter df: remove all terraces above 150m²
    df = df[(df['terrace_area (m²)'].isna()) | (df['terrace_area (m²)'] <= 150)]

    #how many dropped?
    print("dropped",before_drop-df.shape[0],"properties with abnormal terraces")
    return(df)


#stage 0: typeconverion
df = convert_datatypes(df)

#stage 1: drop prices
df = drop_prices(df)

#stage 2: drop subtypes
df = drop_subtypes(df)

#stage 3: drop living areas
df = drop_living_area(df)

#stage 4: drop_terrace areas
df = drop_terrace_area(df)

print("a total of", total_before - df.shape[0], "properties were dropped")

#write to output file
df.to_csv("filtered_final_cleaned_data.csv", index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'final_cleaned_data.csv'

# DISTRIBUTIONS & OUTLIER ANALYSIS

# missing values filtered dataset


In [ ]:
# Check for missing values (FARANGES)
missing_data = df.isnull().sum()

# Put the missing values per column in percentage
missing_summary = pd.DataFrame({
    'Missing Values': df.isnull().sum(),
    'Percentage (%)': (df.isnull().sum() / len(df)) * 100
})

# Round off the percentages
missing_summary['Percentage (%)'] = missing_summary['Percentage (%)'].round(0)
display(missing_summary)

# distribution of type

In [ ]:
import matplotlib.pyplot as plt

#show bar chart of categories
df['type'].value_counts().plot(kind='barh')
plt.xlabel("Count")
plt.ylabel("Category")
plt.title("Distribution per type")
plt.show()

#how many different values and how much are there?
df["type"].value_counts()

# distribution of subtypes

In [ ]:
#show bar chart of categories
df['subtype'].value_counts().plot(kind='barh')
plt.xlabel("Count")
plt.ylabel("Category")
plt.title("Distribution per subtype")
plt.show()

#how many different values and how much are there?
df["subtype"].value_counts()

# distribution of price

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

fig, ax = plt.subplots(figsize=(6, 8))

# --- Plot boxplot ---
sns.boxplot(data=df, y="price (€)", ax=ax)

# --- Compute whiskers and outliers ---
prices = df["price (€)"]
Q1 = prices.quantile(0.25)
Q3 = prices.quantile(0.75)
IQR = Q3 - Q1
upper_whisker = Q3 + 1.5 * IQR

# Outliers
outliers = prices[prices > upper_whisker]
n_outliers = len(outliers)

# --- Draw ellipse around outliers ---
# Choose ellipse size based on plot y-limits
ymin, ymax = ax.get_ylim()

ellipse_height = (ymax - upper_whisker) * 0.95
ellipse_width = 0.35    # works well for vertical boxplots
ellipse_center_x = 0     # x=0 is where the boxplot is drawn
ellipse_center_y = upper_whisker + ellipse_height / 2

ellipse = patches.Ellipse(
    (ellipse_center_x, ellipse_center_y),
    ellipse_width,
    ellipse_height,
    edgecolor='red',
    facecolor='none',
    linewidth=2
)
ax.add_patch(ellipse)

# --- Add text label ---
ax.text(
    ellipse_center_x,
    ellipse_center_y,
    f"{n_outliers} outliers",
    color='red',
    ha='center',
    va='center',
    fontsize=12,
    weight='bold'
)

plt.show()

In [ ]:
#PRICE for houses less than 1M

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

# make a new df without houses over 1M
df_cheaper = df[(df["price (€)"]) <= 1000000]


# visual upgraded boxplot houses < 1M
ax = sns.boxplot(data=df_cheaper, y="price (€)")

# Extract values
prices = df_cheaper["price (€)"]
median = prices.median()
whisker_low = np.percentile(prices, 25) - 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))
whisker_high = np.percentile(prices, 75) + 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))

# But real whiskers are clipped to the actual data range:
whisker_low = prices[prices >= whisker_low].min()
whisker_high = prices[prices <= whisker_high].max()

#add title
plt.title("Distribution of Price", fontsize=14)

# Annotate
ax.annotate(f"Median: {median:,.0f} €", xy=(0, median),
            xytext=(0.1, median), color="black")

ax.annotate(f"Low whisker: {whisker_low:,.0f} €", xy=(0, whisker_low),
            xytext=(0.1, whisker_low), color="blue")

ax.annotate(f"High whisker: {whisker_high:,.0f} €", xy=(0, whisker_high),
            xytext=(0.1, whisker_high), color="green")

plt.show()

#plot the distributions without houses over 1M
df_cheaper = df[(df["price (€)"]) <= 1000000]
sns.histplot(df_cheaper["price (€)"], kde=True, bins=100)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))
#plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(500_000))
plt.xlabel("Price (€)")
plt.title('Price Distribution')
plt.ylabel('Count')
plt.show()

#checking for outliers of price (quantitavely IQR and z-scores)
#IQR
Q1 = df["price (€)"].quantile(0.25)
Q3 = df["price (€)"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#1K houses are outside of the current IQR
outliers = df[(df["price (€)"] < lower) | (df["price (€)"] > upper)]
display(outliers.head())
print(len(outliers))






# distribution of bedrooms

In [ ]:
#how many categories and how much?
display(df["number_of_bedrooms"].value_counts())
#show 50 bedroom house
df_fifty_bed = df[(df["number_of_bedrooms"]) == 50]
display(df_fifty_bed)
print("https://immovlan.be/nl/detail/kasteel/te-koop/3840/borgloon/rbu6280")
#show zero bedroom house:
df_zero_bed = df[(df["number_of_bedrooms"]) == 0]
display(df_zero_bed)

# distribution of living area

In [ ]:
#Living area  -> analysis : 
# visual via boxplot 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

sns.boxplot(data=df, y="living_area (m²)")
plt.show()

# Plot the distribution (KDE + histogram)
sns.histplot(df["living_area (m²)"], kde=True, bins=100)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))
#plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(500_000))
plt.xlabel("living_area (m²)")
plt.title('Area Distribution')
plt.ylabel('Count')
plt.show()

# visual upgraded boxplot houses with living area < 500m² 
ax = sns.boxplot(data=df_cheaper, y="living_area (m²)")

# Extract values
prices = df_cheaper["living_area (m²)"]
median = prices.median()
whisker_low = np.percentile(prices, 25) - 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))
whisker_high = np.percentile(prices, 75) + 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))

# But real whiskers are clipped to the actual data range:
whisker_low = prices[prices >= whisker_low].min()
whisker_high = prices[prices <= whisker_high].max()

# Annotate
ax.annotate(f"Median: {median:,.0f} m²", xy=(0, median),
            xytext=(0.1, median), color="black")

ax.annotate(f"Low whisker: {whisker_low:,.0f} m²", xy=(0, whisker_low),
            xytext=(0.1, whisker_low), color="blue")

ax.annotate(f"High whisker: {whisker_high:,.0f} m²", xy=(0, whisker_high),
            xytext=(0.1, whisker_high), color="green")

plt.show()

#replot the distribution without properties with areas over 500m²
df_cheaper = df[(df["living_area (m²)"]) <= 500]
sns.histplot(df_cheaper["living_area (m²)"], kde=True, bins=50)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(50))
plt.xlabel("living_area (m²)")
plt.title('Area Distribution')
plt.ylabel('Count')
plt.show()

#checking for outliers of living area (quantitavely IQR and z-scores)
#IQR
Q1 = df["living_area (m²)"].quantile(0.25)
Q3 = df["living_area (m²)"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#1K houses are outside of the current IQR
outliers = df[(df["living_area (m²)"] < lower) | (df["living_area (m²)"] > upper)]
display(outliers.head())
print(len(outliers))


#biggest living areas top 5 -> DROP the biggest one here
df_sorted = df.sort_values(by="living_area (m²)", ascending=False).head()
display(df_sorted)

# distribution of equiped kitchen

In [ ]:

df['equiped_kitchen (yes:1, no:0)'].value_counts().plot(kind='barh')
plt.xlabel("Count")
plt.ylabel("Category")
plt.title("Distribution equiped kitchen")
plt.show()

#how many different values and how much are there?
df["equiped_kitchen (yes:1, no:0)"].value_counts()

# distibrution of furnished, open fire and terrace

In [ ]:
#how many different values and how much are there?
df["furnished (yes:1, no:0)"].value_counts()

#how many different values and how much are there?
df["open_fire (yes:1, no:0)"].value_counts()

#how many different values and how much are there?
df["terrace (yes:1, no:0)"].value_counts()

# distribution of terrace area

In [ ]:
#Terrace area  -> analysis : 
# visual via boxplot 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

sns.boxplot(data=df, y="terrace_area (m²)")
plt.show()

# visual upgraded boxplot houses with terrace area 
ax = sns.boxplot(data=df, y="terrace_area (m²)")

# Extract values
prices = df["terrace_area (m²)"]
median = prices.median()
whisker_low = np.percentile(prices, 25) - 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))
whisker_high = np.percentile(prices, 75) + 1.5 * (np.percentile(prices, 75) - np.percentile(prices, 25))

# But real whiskers are clipped to the actual data range:
whisker_low = prices[prices >= whisker_low].min()
whisker_high = prices[prices <= whisker_high].max()

# Annotate
ax.annotate(f"Median: {median:,.0f} m²", xy=(0, median),
            xytext=(0.1, median), color="black")

"""ax.annotate(f"Low whisker: {whisker_low:,.0f} m²", xy=(0, whisker_low),
            xytext=(0.1, whisker_low), color="blue")

ax.annotate(f"High whisker: {whisker_high:,.0f} m²", xy=(0, whisker_high),
            xytext=(0.1, whisker_high), color="green")"""

plt.show()

# Plot the distribution (KDE + histogram)
sns.histplot(df["terrace_area (m²)"], kde=True, bins=100)
plt.gca().xaxis.set_major_locator(ticker.MaxNLocator(nbins=15))
#plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(500_000))
plt.xlabel("terrace_area (m²)")
plt.title('Area Distribution')
plt.ylabel('Count')
plt.show()

#checking for outliers of living area (quantitavely IQR and z-scores)
#IQR
Q1 = df["terrace_area (m²)"].quantile(0.25)
Q3 = df["terrace_area (m²)"].quantile(0.75)
IQR = Q3 - Q1

lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#1K houses are outside of the current IQR
outliers = df[(df["terrace_area (m²)"] < lower) | (df["terrace_area (m²)"] > upper)]
display(outliers.head())
print(len(outliers))

#biggest terrace areas top 5 -> DROP the biggest one here
df_sorted = df.sort_values(by="terrace_area (m²)", ascending=False).head()
display(df_sorted)

# distribution of garden, number of facades, swimming pool and state of building

In [ ]:
df['garden (yes:1, no:0)'].value_counts().plot(kind='barh')
plt.xlabel("Count")
plt.ylabel("Category")
plt.title("Distribution garden")
plt.show()

#how many different values and how much are there?
df["garden (yes:1, no:0)"].value_counts()

#how many different values and how much are there?
df["number_facades"].value_counts()

#how many different values and how much are there?
df["swimming_pool (yes:1, no:0)"].value_counts()

#how many different values and how much are there?
display(df["state_of_building"].value_counts())
demolish = df[(df["state_of_building"] == "To demolish")] 
display(demolish)

# CORRELATION ANALYSIS BETWEEN PRICE AND GDP PER CAPITA (PER PROVINCE)


In [ ]:
# Correlation analysis:

"""Positive correlation (+1 for perfect correlation): the two variables change in the same direction.
Negative correlation (-1): the two variables change in opposite directions.
No correlation (0): there is no association or relevant relationship between the two variables."""

# Select numeric values
numeric_df = df.select_dtypes(include="number")

# Correlation matrix for numeric columns

corr_matrix = numeric_df.corr()
corr_matrix['price (€)'].sort_values(ascending=False)

# Visualize in a heatmap for the numeric columns (too dense)
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, cmap='coolwarm', annot=False)
plt.title("Correlation Heatmap")
plt.show()





In [ ]:
#Correlation between two numeric variables: property price and GDP ome per capita in €:
# Source for income per capita (gdp) per province in Belgium: Eurostat https://ec.europa.eu/eurostat/databrowser/view/tgs00003/default/table?lang=en

# dictionnary gdp per capita shown per province

gdp_per_province = {"Antwerp": 114382, 
            "East-Flanders": 74452, 
            "Limburg": 36726 , 
            "West-Flanders": 60931, 
            "Flemish-Brabant": 69558, 
            "Liège": 41196, 
            "Brabant-Wallon": 26475, 
            "Namur": 17623 ,
            "Hainaut": 42079,
            "Luxembourg": 9424,
            "Brussels": 103285}

# 1) Convert the dictionary to a DataFrame
gdp_df = pd.DataFrame(list(gdp_per_province.items()), columns=["province", "GDP"])

# 2️) Display the DataFrame to check it looks right
print(gdp_df)

# 3️) Save it as a CSV file
gdp_df.to_csv("gdp_per_province.csv", index=False)

print("CSV file 'gdp_per_province.csv' has been created!")

display(gdp_df)



In [ ]:
"""We need to add a "province" column to our filtered_final_cleaned_data.csv file. 
Since we only have postal codes, we'll first need to map them to their provinces. 
We'll use a dictionary mapping postal codes to provinces."""

# Load data
df = pd.read_csv("filtered_final_cleaned_data.csv")

# Define postal code ranges per province
postal_to_province = {
    "Antwerp": range(2000, 3000),
    "East-Flanders": range(9000, 10000),
    "West-Flanders": range(8000, 9000),
    "Flemish-Brabant": list(range(1500, 2000)) + list(range(3000, 3500)),
    "Brussels": range(1000, 1300),
    "Limburg": range(3500, 4000),
    "Liège": range(4000, 5000),
    "Namur": range(5000, 6000),
    "Hainaut": list(range(6000, 6600)) + list(range(7000, 8000)),
    "Luxembourg": range(6600, 7000),
    "Brabant-Wallon": range(1300, 1500)
}

# Helper function to find province for each postal code
def get_province(postal_code):
    try:
        postal_code = int(postal_code)
        for province, codes in postal_to_province.items():
            if postal_code in codes:
                return province
        return "Unknown"
    except:
        return "Unknown"


# Apply the function to create a new column called "province" with the province names based on the postal codes.
df["province"] = df["postal_code"].apply(get_province)

print(df[["postal_code", "province"]].head())

# display(df.head())

# Save the updated dataset
df.to_csv("filtered_final_cleaned_data.csv", index=False)
print("New CSV saved with 'province' column!")

# Load the new dataset with province column
df = pd.read_csv("filtered_dataset_int.csv")


In [ ]:
# Calculate the average price per province 

avg_price = df.groupby("province", as_index=False)["price (€)"].mean()
avg_price.rename(columns={"price (€)": "avg_price"}, inplace=True)
avg_price["avg_price"] = avg_price["avg_price"].round(0).astype(int)

display(avg_price)


# Merge with GDP dataset

gdp_df = pd.read_csv("gdp_per_province.csv")
df_merged = pd.merge(avg_price, gdp_df, on="province")

display(df_merged)

In [ ]:
# Check the correlation value between average price per province and gdp per province
corr_value = df_merged["avg_price"].corr(df_merged["GDP"])
print(f"Correlation between average property price per province and GDP per capita per province: {corr_value:.2f}")

In [ ]:
# Visualize the correlation between gdp per capita and price in scatter plot

import matplotlib.pyplot as plt

#Compute the correlation once
corr = df_merged["GDP"].corr(df_merged["avg_price"])

plt.figure(figsize=(10,7))
plt.scatter(df_merged["GDP"], df_merged["avg_price"], alpha=0.6, )  # alpha for transparency

# Add province names as labels
for i, row in df_merged.iterrows():
    plt.text(row["GDP"], row["avg_price"], row["province"], fontsize=9, ha='center', va='bottom')

plt.xlabel("GDP per capita per province (€)")
plt.ylabel("Average Property price (€)")
plt.title("Correlation between GDP per capita and Average Property Price per Province")
plt.grid(True,linestyle='--', alpha=0.5)

# Add correlation coefficient on the plot
plt.text(
    df_merged["GDP"].min(),
    df_merged["avg_price"].max() * 0.95,
    f"Correlation: {corr:.2f}",
    fontsize=12,
    color="red"
)
plt.show()

# Add a regression (trend) line for clarity to make the relationship clearer (and not just dots), use Seaborn’s regplot:
# the red trendline is showing the correlation, this highlights direction and strength of the correlation 

import seaborn as sns

plt.figure(figsize=(9, 6))
sns.regplot(
    data=df_merged,
    x="GDP",
    y="avg_price",
    scatter_kws={'alpha':0.7, 's':80},
    line_kws={'color':'red', 'lw':2}
)
plt.title("Correlation between GDP per Capita and Property Prices per Province", fontsize=14, weight='bold')

# Add province names as labels
for i, row in df_merged.iterrows():
    plt.text(row["GDP"], row["avg_price"], row["province"], fontsize=9, ha='center', va='bottom')

plt.xlabel("GDP per Capita (€) per province")
plt.ylabel("Average Property Price (€) per Province")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Add correlation coefficient on the plot
plt.text(
    df_merged["GDP"].min(),
    df_merged["avg_price"].max() * 0.95,
    f"Correlation: {corr:.2f}",
    fontsize=12,
    color="red"
)
plt.show()


# Correlation. Assessing the impact of real estate parameters on price

In [3]:
# 1. Import bibliothek and loading data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
from scipy.stats import spearmanr, pointbiserialr, chi2_contingency

df = pd.read_csv("filtered_final_cleaned_data.csv")
df = df.drop(columns=df.columns[:2]) # Delete property_ID; locality_name

# 2. Defining column types
def detect_column_type(series, cat_threshold=15):
    if pd.api.types.is_numeric_dtype(series):
        if series.nunique() == 2:
            return "binary"
        else:
            return "numerical"
    else:
        if series.nunique() == 2:
            return "binary"
        elif series.nunique() <= cat_threshold:
            return "categorical"
        else:
            return "categorical"

column_types = {col: detect_column_type(df[col]) for col in df.columns}

FileNotFoundError: [Errno 2] No such file or directory: 'filtered_final_cleaned_data.csv'

In [ ]:
# 3. Functions for calculating correlation

# Convert a 2-class categorical series to 0/1
def encode_binary(series):

    if series.nunique() != 2:
        return series
    classes = list(series.unique())
    mapping = {classes[0]: 0, classes[1]: 1}
    return series.map(mapping)

# Eta_squared: Numerical vs Categorical
def eta_squared(df, numerical, categorical):
    groups = df.groupby(categorical)[numerical]
    ss_between = sum([len(g) * (g.mean() - df[numerical].mean())**2 for name, g in groups])
    ss_total = sum((df[numerical] - df[numerical].mean())**2)
    return ss_between / ss_total if ss_total != 0 else 0

# Spearman: Numerical vs Numerical
def safe_spearman(a, b):
    a_num = pd.to_numeric(a, errors='coerce')
    b_num = pd.to_numeric(b, errors='coerce')
    mask = a_num.notna() & b_num.notna()
    if mask.sum() < 3:
        return np.nan
    val, _ = spearmanr(a_num[mask], b_num[mask])
    return val

# Point-Biserial: Binary vs Numerical
def safe_pointbiserial(numeric, binary):
    numeric = pd.to_numeric(numeric, errors='coerce')
    binary = pd.to_numeric(binary, errors='coerce')
    mask = numeric.notna() & binary.notna()
    if mask.sum() < 3:
        return np.nan
    try:
        val, _ = pointbiserialr(numeric[mask], binary[mask])
        return val
    except Exception:
        return np.nan

In [ ]:
# 4. Construction of a “correlation vector” with "price (€)"
def correlation_value(x, y, type_x, type_y):
    try:
        # numerical-numerical
        if type_x == "numerical" and type_y == "numerical":
            return safe_spearman(x, y)

        # numerical-binary
        if type_x == "binary" and type_y == "numerical":
            return safe_pointbiserial(encode_binary(x), y)
        if type_x == "numerical" and type_y == "binary":
            return safe_pointbiserial(x, encode_binary(y))

        # numerical-categorical
        if type_x == "numerical" and type_y == "categorical":
            return eta_squared(df, x.name, y.name)

        if type_x == "categorical" and type_y == "numerical":
            return eta_squared(df, y.name, x.name)

        # binary-binary
        if type_x == "binary" and type_y == "binary":
            return phi_coefficient(x, y)

        # categorical-categorical
        if type_x == "categorical" and type_y == "categorical":
            return cramers_v(x, y)

    except Exception:
        return np.nan

    return np.nan

columns = df.columns

target = "price (€)"           # field to compare with
corr_vector = {}               # Let's add the correlations here

for col in df.columns:
    if col == target:
        continue
    t1 = column_types[col]
    t2 = column_types[target]
    corr_vector[col] = correlation_value(df[col], df[target], t1, t2)

corr_vector = pd.Series(corr_vector).sort_values(ascending=False)

print("\nMixed correlation vector with target:", target)
# print(corr_vector)
print(corr_vector.abs().sort_values(ascending=False))

In [ ]:
# 5. Visualization of a mixed matrix
plt.figure(figsize=(6, 12))

# seaborn heatmap expects a 2D matrix → convert Series to DataFrame
corr_df = corr_vector.to_frame(name="correlation")

ax = sns.heatmap(
    corr_df,
    annot=True,
    cmap="coolwarm",
    center=0,
    cbar=True,
    cbar_kws={"orientation": "horizontal"}
)

plt.title(f"Correlation with {target}", pad=50)

# Get existing colorbar
cbar = ax.collections[0].colorbar

# Move ticks to the top of colorbar
cbar.ax.xaxis.set_label_position('top')
cbar.ax.xaxis.tick_top()

# Reposition colorbar under the title
bbox = ax.get_position()
cbar.ax.set_position([bbox.x0, bbox.y1 + 0.03, bbox.width, 0.02])

ax.set_xlabel("")
ax.set_ylabel("Features")
plt.tight_layout(rect=[0, 0, 1, 0.95])  # leave space for colorbar and title

plt.show()

In [ ]:
# 6. Top 5 Strongest Relationships
top5 = corr_vector.abs().sort_values(ascending=False).head(5)
top5 = corr_vector.loc[top5.index]
top5

# 7. Visualization of the TOP-5 strongest connections
top5 = corr_vector.abs().sort_values(ascending=False).head(5)
top5 = corr_vector.loc[top5.index]   # restore ± sign

print("\nTOP-5 strongest correlations with price (€):")
print(top5)

# --- Visualization of TOP-5 as individual plots ---
plt.figure(figsize=(15, 20))

for i, feature in enumerate(top5.index, 1):
    plt.subplot(5, 1, i)

    ftype = column_types[feature]

    # Numerical vs Numerical → SCATTERPLOT + TREND LINE
    if ftype == "numerical":
        # scatter
        sns.scatterplot(data=df, x=feature, y=target, alpha=0.5)
        
        # ---- trend line ----
        # drop NaN
        x = df[feature].astype(float)
        y = df[target].astype(float)
        mask = x.notna() & y.notna()

        if mask.sum() > 2:
            # linear regression
            slope, intercept = np.polyfit(x[mask], y[mask], 1)
            x_line = np.linspace(x.min(), x.max(), 100)
            y_line = slope * x_line + intercept

            plt.plot(x_line, y_line, linewidth=2)

        plt.title(f"{feature} vs {target} (numerical → scatter + trend line)")
        plt.xlabel(feature)
        plt.ylabel(target)

    # Binary vs Numerical → BOXPLOT (trend line NOT applicable)
    elif ftype == "binary":
        sns.boxplot(data=df, x=feature, y=target)
        plt.title(f"{feature} vs {target} (binary → boxplot)")
        plt.xlabel(feature)
        plt.ylabel(target)

    # Categorical vs Numerical → BOXPLOT (trend line NOT applicable)
    elif ftype == "categorical":
        # Если категорий слишком много — показывать только top-10 самых частых
        if df[feature].nunique() > 10:
            top_categories = df[feature].value_counts().head(10).index
            sub_df = df[df[feature].isin(top_categories)]
            sns.boxplot(data=sub_df, x=feature, y=target)
            plt.title(f"{feature} vs {target} (categorical, top-10 categories)")
        else:
            sns.boxplot(data=df, x=feature, y=target)
            plt.title(f"{feature} vs {target} (categorical → boxplot)")

        plt.xlabel(feature)
        plt.ylabel(target)

    plt.tight_layout()

plt.show()

In [ ]:
"""Now do the same, but for the median price per province instead of the average price"""

# Calculate the median price per province 

median_price = df.groupby("province", as_index=False)["price (€)"].median()
median_price.rename(columns={"price (€)": "median_price"}, inplace=True)
median_price["median_price"] = median_price["median_price"].round(0).astype(int)

display(median_price)

# Merge with GDP dataset

gdp_df = pd.read_csv("gdp_per_province.csv")
df_merged = pd.merge(median_price, gdp_df, on="province")

display(df_merged)


In [ ]:
# Check the correlation value between median price per province and gdp per province
corr_value = df_merged["median_price"].corr(df_merged["GDP"])
print(f"Correlation between median property price per province and GDP per capita per province: {corr_value:.2f}")

In [ ]:
# Visualize the correlation between gdp per capita and median price in scatter plot

import matplotlib.pyplot as plt

#Compute the correlation once
corr = df_merged["GDP"].corr(df_merged["median_price"])


plt.figure(figsize=(10,7))
plt.scatter(df_merged["GDP"], df_merged["median_price"], alpha=0.6, )  # alpha for transparency

# Add province names as labels
for i, row in df_merged.iterrows():
    plt.text(row["GDP"], row["median_price"], row["province"], fontsize=9, ha='center', va='bottom')

plt.xlabel("GDP per capita per province (€)")
plt.ylabel("Median Property price (€)")
plt.title("Correlation between GDP per capita and Median Property Price per Province")
plt.grid(True,linestyle='--', alpha=0.5)

# Add correlation coefficient on the plot
plt.text(
    df_merged["GDP"].min(),
    df_merged["median_price"].max() * 0.95,
    f"Correlation: {corr:.2f}",
    fontsize=12,
    color="red"
)
plt.show()

# Add a regression (trend) line for clarity to make the relationship clearer (and not just dots), use Seaborn’s regplot:
# the red trendline is showing the correlation, this highlights direction and strength of the correlation 

import seaborn as sns

plt.figure(figsize=(9, 6))
sns.regplot(
    data=df_merged,
    x="GDP",
    y="median_price",
    scatter_kws={'alpha':0.7, 's':80},
    line_kws={'color':'red', 'lw':2}
)
plt.title("Correlation between GDP per Capita and Median Property Prices per Province", fontsize=14, weight='bold')

# Add province names as labels
for i, row in df_merged.iterrows():
    plt.text(row["GDP"], row["median_price"], row["province"], fontsize=9, ha='center', va='bottom')

plt.xlabel("GDP per Capita (€) per province")
plt.ylabel("Median Property Price (€) per Province")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()

# Add correlation coefficient on the plot
plt.text(
    df_merged["GDP"].min(),
    df_merged["median_price"].max() * 0.95,
    f"Correlation: {corr:.2f}",
    fontsize=12,
    color="red"
)
plt.show()


Conclusion for correlation between gdp per capita and median property prices: 
- there is a positive correlation (correlation value 0,48) between socioeconomic indicators like gdp per capita and property prices. 
- 0.48 = moderate positive correlation => It is meaningful, but not strong.
-The variables move somewhat together, but many exceptions exist.
- Our result of 0.48 means that there is a real connection between wealth of a province and housing prices.
But GDP per capita alone cannot predict housing prices accurately.
- How to interpret it properly:
A correlation of 0.48 implies:
GDP per capita explains around 23% of the variation in property prices
(because correlation² = 0.48² ≈ 0.23 → “explained variance”).
So:
23% of the differences in property prices across provinces
could be associated with GDP per capita.
The other 77% is explained by other factors such as supply/demand, population density, region attractiveness, nearby cities, salary distribution, housing supply, etc.
So GDP per capita has influence — but it’s not the only important factor.

Are the values (0.48 and 0.50) realistic? Yes.
In Belgium:
Property prices correlate with wealth, but not perfectly.
Brussels is high GDP but also has many cheap apartments.
Wallonia has low GDP but also some pockets of expensive rural houses.
Flanders has higher GDP but also large variation inside provinces.
Typical correlations found in academic or market studies range from 0.3 to 0.7, depending heavily on the variable definition.

A higher correlation is definitely possible — but only under certain conditions, like:
A different data source (more complete, more luxury listings)
Prices aggregated differently (per type of property, per year, per municipality)
Cleaned outliers differently (e.g., removing €30k listings that distort the median)
Aggregated over a different time window
Used a dataset with more balanced counts per province

# RANKINGS: What are the least/most expensive provinces and municipalities in Belgium/Wallonia/Flanders? (in terms of price per m², average price, and median price)

In [ ]:
""" - Average price per province and per municipality  => draw conclusion in terms of least/most expensive
    - Median price per province and per municipality => draw conclusion in terms of least/most expensive
    - Price per m² per province and municipality => draw conclusion in terms of least/most expensive"""

# 1) Create a "price per m²” column (we need this for all ranking questions)
# Take the living area in m² to make the calculation (most common - later we can add terrace area annd garden area)

df = pd.read_csv("filtered_final_cleaned_data.csv")

df['price_per_m2'] = df['price (€)'] / df['living_area (m²)']
df['price_per_m2'] = df['price_per_m2'].round(0)


df.to_csv("filtered_final_cleaned_data.csv", index=False)

df[['price (€)', 'living_area (m²)', 'price_per_m2']].head()



In [ ]:
""" 2) Calculate stats per province: Average&median price and average&median price per m²:
This answers:
Most/least expensive province by average price
Most/least expensive province by median price
Most/least expensive province by price per m²"""

province_stats = df.groupby('province').agg({
    'price (€)': ['mean', 'median'],
    'price_per_m2': ['mean', 'median']
})

province_stats = province_stats.round(0).astype("Int64")              # Round the results to integers

province_stats.sort_values(('price (€)','mean'), ascending=False)     # Most expensive avg price
province_stats.sort_values(('price (€)','mean'), ascending=True)      # Least expensive avg price

province_stats.sort_values(('price (€)','median'), ascending=False)     # Most expensive median price
province_stats.sort_values(('price (€)','median'), ascending=True)      # Least expensive median price

province_stats.sort_values(('price_per_m2','mean'), ascending=False)
province_stats.sort_values(('price_per_m2','mean'), ascending=True)

display(province_stats.sort_values(('price (€)','mean'), ascending=False))



In [ ]:
"""3) Calculate stats per municipality: same approach as for the stats per province above.
This answers:
Most/least expensive municipality (avg price)
Most/least expensive municipality (median price)
Most/least expensive municipality (price/m²)"""


# Remove leftover municipality columns from earlier merges
df = df.drop(columns=[col for col in df.columns if col.startswith("municipality")],
             errors="ignore")

# Load postal codes from separate csv file with official postal codes and municipality names

postal_codes = pd.read_csv(
    "postal-codes-belgium.csv",
    sep=";",
    engine="python",
    on_bad_lines="skip",
    usecols=["Postal Code", "Municipality name (Dutch)", "Municipality name (French)"])

postal_codes["municipality"] = (
    postal_codes["Municipality name (Dutch)"].fillna(postal_codes["Municipality name (French)"])
)


# Merge the municipality column into your property dataset
df = df.merge(
    postal_codes[["Postal Code", "municipality"]],
    left_on="postal_code",
    right_on="Postal Code",
    how="left"
)

# Drop the duplicate column 
df = df.drop(columns=["Postal Code"])

df.to_csv("filtered_final_cleaned_data.csv", index=False)

# Count listings per municipality
counts = df['municipality'].value_counts()

# Keep only municipalities with >= 10 listings
valid_municipalities = counts[counts >= 10].index

# Filter your main df
df_filtered = df[df['municipality'].isin(valid_municipalities)]


# Stats per municipality 

municipality_stats = df_filtered.groupby('municipality').agg({
    'price (€)': ['mean', 'median'],
    'price_per_m2': ['mean', 'median']
})
municipality_stats = municipality_stats.round(0).astype("Int64")            # Round the results to integers


municipality_stats.sort_values(('price (€)','mean'), ascending=False)       # Top 10 expensive avg
municipality_stats.sort_values(('price (€)','mean'), ascending=True)        # Top 10 cheap avg

municipality_stats.sort_values(('price (€)','median'), ascending=False)     # Top 10 expensive median
municipality_stats.sort_values(('price (€)','median'), ascending=True)      # Top 10 cheap median

municipality_stats.sort_values(('price_per_m2','mean'), ascending=False)
municipality_stats.sort_values(('price_per_m2','mean'), ascending=True)

display(municipality_stats.sort_values(('price (€)','mean'), ascending=False).head(10))


In [ ]:
"""4) Filter by region (Wallonia vs Flanders vs Brussels)"""

# Create a region column one time

df = pd.read_csv("filtered_final_cleaned_data.csv")

def get_region(province):
    flanders = ['Antwerp', 'Limburg', 'East-Flanders', 'West-Flanders', 'Flemish-Brabant']
    wallonia = ['Hainaut', 'Liège', 'Luxembourg', 'Namur', 'Brabant-Wallon']
    brussels = ['Brussels']

    if province in flanders:
        return 'Flanders'
    if province in wallonia:
        return 'Wallonia'
    return 'Brussels'

df['region'] = df['province'].apply(get_region)

df.to_csv("filtered_final_cleaned_data.csv", index=False)

# Stats per region

region_stats = df.groupby('region').agg({
    'price (€)': ['mean', 'median'],
    'price_per_m2': ['mean', 'median']
})

region_stats = region_stats.round(0).astype("Int64")  

display(region_stats)

"""Conclusion: Brussels is the region with the highest average prices and highest average price per m2 """

In [ ]:
# Visualisations of average prices and prices per m2 per province and per municipality

# Most Expensive Provinces Ranking (Average Price)

top10_expensive_provinces = (
    df.groupby('province')['price (€)']
      .mean()
      .sort_values(ascending=False)
)

plt.figure(figsize=(10,6))
plt.barh(top10_expensive_provinces.index, top10_expensive_provinces.values)
plt.gca().invert_yaxis()
plt.title("Most Expensive Provinces Ranking (Average Price)")
plt.xlabel("Average Price (€)")
plt.tight_layout()
plt.show()

# Top Ten Most Expansive Municipalities (Average Price)

top10_expensive_municipalities = (
    df_filtered.groupby('municipality')['price (€)']
      .mean()
      .sort_values(ascending=False)
      .head(10)
)

plt.figure(figsize=(10,6))
plt.barh(top10_expensive_municipalities.index, top10_expensive_municipalities.values)
plt.gca().invert_yaxis()
plt.title("Top 10 Most Expensive Municipalities (Average Price)")
plt.xlabel("Average Price (€)")
plt.tight_layout()
plt.show()

# Median Price per Province

median_province = (
    df.groupby('province')['price (€)']
      .median()
      .sort_values(ascending=False)
)

plt.figure(figsize=(10,6))
plt.barh(median_province.index, median_province.values)
plt.gca().invert_yaxis()
plt.title("Median Price per Province")
plt.xlabel("Median Price (€)")
plt.tight_layout()
plt.show()

# Top 10 most expensive median price per Municipality


top10_expensive_municipalities_median = (
    df_filtered.groupby('municipality')['price (€)']
      .median()
      .sort_values(ascending=False)
      .head(10)
)

plt.figure(figsize=(9,7))
plt.barh(top10_expensive_municipalities_median.index, top10_expensive_municipalities_median.values)
plt.gca().invert_yaxis()
plt.title("Top 10 Most Expensive Municipalities (Median Price)")
plt.xlabel("Median Price (€)")
plt.tight_layout()
plt.show()


# Price per m² per Province (Average)

df['price_per_m2'] = df['price (€)'] / df['living_area (m²)']

avg_ppm_province = (
    df.groupby('province')['price_per_m2']
      .mean()
      .sort_values(ascending=False)
)

plt.figure(figsize=(10,6))
plt.barh(avg_ppm_province.index, avg_ppm_province.values)
plt.gca().invert_yaxis()
plt.title("Average Price per m² per Province")
plt.xlabel("Price per m² (€)")
plt.tight_layout()
plt.show()

# Price per m² per Municipality

top10_ppm = (
    df_filtered.groupby('municipality')['price_per_m2']
      .mean()
      .sort_values(ascending=False)
      .head(10)
)

plt.figure(figsize=(10,6))
plt.barh(top10_ppm.index, top10_ppm.values)
plt.gca().invert_yaxis()
plt.title("Top 10 Most Expensive Municipalities (Price per m²)")
plt.xlabel("Price per m² (€)")
plt.tight_layout()
plt.show()







# Correlation Analysis and price comparison between different variables

In [ ]:
import pandas as pd
import plotly.express as px
import statsmodels.api as sm
from statsmodels.formula.api import ols


def clean_and_load():

    df = pd.read_csv("filtered_final_cleaned_data.csv")

    # converting float to int
    df = df.apply(lambda x: x.astype("Int64") if x.dtype == float and (x.dropna() % 1 == 0).all() else x)

    # converting objects to strings
    df['property_ID'] = df['property_ID'].astype('string')
    df['locality_name'] = df['locality_name'].astype('string')
    df['type'] = df['type'].astype('category')
    df['subtype'] = df['subtype'].astype('category')
    df['state_of_building'] = df['state_of_building'].astype('category')
    df['postal_code'] = df['postal_code'].astype('category')
    
    # removing properties that do not have the price
    df = df.dropna(subset=["price (€)"])
    #display(df.dtypes)
    df.to_csv("cleaned_data_int.csv", index=False)
    
    return df

df = clean_and_load()

# --- Display results in notebook ---
display(df.dtypes)   # Shows column types         

In [ ]:
def price_corr_postal_code():

    df = clean_and_load()
    
    # ANOVA: treats postal_code as categorical
    model = ols('Q("price (€)") ~ C(postal_code)', data=df).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)  

    # Group by location to compute median price and count of listings
    median_price_by_location = df.groupby("postal_code", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_location["median_price"] = median_price_by_location["median_price"].round(2)

    # scatter plot
    fig = px.scatter(median_price_by_location, x="postal_code", y="median_price", size="num_listings", title="Price variation based on Postal Code", labels={"postal_code": "Postal Code", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.show()

    # print(median_price_by_location.head())
    return df

df = price_corr_postal_code()

# The F-statistic is 6.579, and p-value is essentially zero.
# This means there is a statistically significant difference in price across postal codes.

In [ ]:
def price_corr_number_of_bedrooms():
    df = clean_and_load()

    # Compute correlation between number of bedrooms and property price
    corr = df["number_of_bedrooms"].corr(df["price (€)"])
    print(f"Correlation between number of bedrooms and price: {corr:.2f}")

    # Group by number of bedrooms to compute median price and count of listings
    median_price_by_bedrooms = df.groupby("number_of_bedrooms", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_bedrooms["median_price"] = median_price_by_bedrooms["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_bedrooms, x="number_of_bedrooms", y="median_price", text="median_price", title="Price variation based on number of bedrooms", labels={"number_of_bedrooms": "Number of Bedrooms", "median_price": "Median Price (€)"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_bedrooms["median_price"].max()*1.1])
    fig.update_xaxes(dtick=1)
    fig.show()

    # print(median_price_by_bedrooms.head())
    return df

df = price_corr_number_of_bedrooms()
# print(df.columns.tolist())

In [ ]:
def price_corr_living_area():
    df = clean_and_load()

    # Compute correlation between living area and property price
    corr = df["living_area (m²)"].corr(df["price (€)"])
    print(f"Correlation between living area and price: {corr:.2f}")

    # Group by living area to compute median price and count of listings
    median_price_by_living_area = df.groupby("living_area (m²)", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_living_area["median_price"] = median_price_by_living_area["median_price"].round(2)

    # scatter plot
    fig = px.scatter(median_price_by_living_area, x="living_area (m²)", y="median_price", size="num_listings", trendline = "ols", title="Price variation based on living area (m²)", labels={"living_area (m²)": "Living area (m²)", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.show()

    print(median_price_by_living_area.head())
    return df

df = price_corr_living_area()
# print(df.columns.tolist())

In [ ]:
def price_corr_swimming_pool():
    df = clean_and_load()

    # Compute correlation between presence of swimming pool (yes:1, no:0) and property price
    corr = df["swimming_pool (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between swimming pool and price: {corr:.2f}")
    df["swimming_pool_str"] = df["swimming_pool (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OTUCOME --> Houses with a swimming pool tend to be slightly more expensive on average, but the effect is not strong."""

    # Group by swimming pool presence to compute median price and count of listings
    median_price_by_swimming_pool = df.groupby("swimming_pool_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_swimming_pool["median_price"] = median_price_by_swimming_pool["median_price"].round(2)
    
    # bar plot
    fig = px.bar(median_price_by_swimming_pool, x="swimming_pool_str", y="median_price", color="num_listings", text="median_price", title="Price variation based on swimming pool", labels={"swimming_pool_str": "Swimming Pool", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_swimming_pool["median_price"].max()*1.1])
    fig.show()

    print(median_price_by_swimming_pool.head())
    return df

df = price_corr_swimming_pool()
# print(df.columns.tolist())

In [ ]:
def price_corr_garden():
    df = clean_and_load()

    # Compute correlation between presence of garden (yes:1, no:0) and property price
    corr = df["garden (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between garden and price: {corr:.2f}")
    df["garden_str"] = df["garden (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OUTCOME: Houses with gardens tend to be slightly more expensive on average, but the effect is not strong."""
    
    # Group by garden presence to compute median price and count of listings
    median_price_by_garden = df.groupby("garden_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_garden["median_price"] = median_price_by_garden["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_garden, x="garden_str", y="median_price", color="num_listings", text="median_price", title="Price Variation by Garden Presence", labels={"garden_str": "Garden", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_garden["median_price"].max()*1.1])
    fig.show()

    print(median_price_by_garden.head())
    return df

df = price_corr_garden()
# print(df.columns.tolist())

In [ ]:
def price_corr_number_of_facades():
    df = clean_and_load()

    # Compute correlation between number of facades and property price
    corr = df["number_facades"].corr(df["price (€)"])
    print(f"Correlation between number of facades and price: {corr:.2f}")
    
    # Group by number of bedrooms to compute median price and count of listings
    median_price_by_facades = df.groupby("number_facades", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_facades["median_price"] = median_price_by_facades["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_facades, x="number_facades", y="median_price", text="median_price", title="Price variation based on number facades", labels={"number_facades": "Number Facades", "median_price": "Median Price (€)"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_facades["median_price"].max()*1.1])
    fig.update_xaxes(dtick=1)
    fig.show()

    return df

df = price_corr_number_of_facades()

In [ ]:
def price_corr_terrace_area():
    df = clean_and_load()

    # Compute correlation between terrace area and property price
    corr = df["terrace_area (m²)"].corr(df["price (€)"])
    print(f"Correlation between terrace area and price: {corr:.2f}")

    # Group by living area to compute median price and count of listings
    median_price_by_terrace_area = df.groupby("terrace_area (m²)", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_terrace_area["median_price"] = median_price_by_terrace_area["median_price"].round(2)

    # scatter plot
    fig = px.scatter(median_price_by_terrace_area, x="terrace_area (m²)", y="median_price", trendline = "ols", size="num_listings", title="Price variation based on terrace area", labels={"terrace_area (m²)": "Terrace area (m²)", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.show()

    return df

df = price_corr_terrace_area()

In [ ]:
def price_corr_terrace():
    df = clean_and_load()

    # Compute correlation between presence of terrace (yes:1, no:0) and property price
    corr = df["terrace (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between terrace and price: {corr:.2f}")
    df["terrace_str"] = df["terrace (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OUTCOME: Houses with terraces tend to be slightly more expensive on average, but the effect is not strong."""

    # Group by terrace presence to compute median price and count of listings
    median_price_by_terrace = df.groupby("terrace_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_terrace["median_price"] = median_price_by_terrace["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_terrace, x="terrace_str", y="median_price", color="num_listings", text="median_price", title="Price Variation by Terrace Presence", labels={"terrace_str": "Terrace", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_terrace["median_price"].max()*1.1])
    fig.show()

    return df

df = price_corr_terrace()

In [ ]:
def price_corr_open_fire():
    df = clean_and_load()

    # Compute correlation between presence of open fire (yes:1, no:0) and property price
    corr = df["open_fire (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between open fire and price: {corr:.2f}")
    df["open_fire_str"] = df["open_fire (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OUTCOME: Houses with open fire tend to be slightly more expensive on average, but the effect is not strong."""

    # Group by terrace presence to compute median price and count of listings
    median_price_by_open_fire = df.groupby("open_fire_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_open_fire["median_price"] = median_price_by_open_fire["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_open_fire, x="open_fire_str", y="median_price", color="num_listings", text="median_price", title="Price Variation by open fire", labels={"open_fire_str": "Open Fire", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_open_fire["median_price"].max()*1.1])
    fig.show()

    return df

df = price_corr_open_fire()

In [ ]:
def price_corr_furnished():
    df = clean_and_load()

    # Compute correlation between furnished property and property price
    corr = df["furnished (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between furnished property and price: {corr:.2f}")
    df["furnished_str"] = df["furnished (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OUTCOME: A correlation that is close to zero suggests that — statistically —
    whether a property is furnished or not does not influence the price in the dataset."""

    # Group by terrace presence to compute median price and count of listings
    median_price_by_furnished = df.groupby("furnished_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_furnished["median_price"] = median_price_by_furnished["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_furnished, x="furnished_str", y="median_price", color="num_listings", text="median_price", title="Price Variation depending if the property is furnished", labels={"furnished_str": "Furnished", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_furnished["median_price"].max()*1.1])
    fig.show()

    return df

df = price_corr_furnished()


In [ ]:
def price_corr_equipped_kitchen():
    df = clean_and_load()

    # Compute correlation between equipped kitchen and property price
    corr = df["equiped_kitchen (yes:1, no:0)"].corr(df["price (€)"])
    print(f"Correlation between equipped kitchen and price: {corr:.2f}")
    df["equipped_kitchen_str"] = df["equiped_kitchen (yes:1, no:0)"].map({0: "No", 1: "Yes"})

    """ OUTCOME: A correlation that is close to zero suggests that — statistically — 
    whether a property has an equipped kitchen or not does not influence the price in the dataset"""

    # Group by terrace presence to compute median price and count of listings
    median_price_by_equipped_kitchen = df.groupby("equipped_kitchen_str", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_equipped_kitchen["median_price"] = median_price_by_equipped_kitchen["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_equipped_kitchen, x="equipped_kitchen_str", y="median_price", color="num_listings", text="median_price", title="Price Variation depending if the property has equipped kitchen", labels={"equipped_kitchen_str": "Equipped Kitchen", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_equipped_kitchen["median_price"].max()*1.1])
    fig.show()

    return df

df = price_corr_furnished()

In [ ]:
def price_corr_state_of_building():
    df = clean_and_load()
    
    # ANOVA: treats state of building as categorical
    model = ols('Q("price (€)") ~ C(state_of_building)', data=df).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)

    """ The ANOVA shows that the state of the building has a highly significant effect on property price (p ≈ 2.3e-112). 
    However, the effect size is modest, meaning building condition explains only a small portion of the overall price variation."""  

    # Group by location to compute median price and count of listings
    median_price_by_state_of_building = df.groupby("state_of_building", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_state_of_building["median_price"] = median_price_by_state_of_building["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_state_of_building, x="state_of_building", y="median_price", color="num_listings", text="median_price", title="Price Variation depending on the state of building", labels={"state_of_building": "State of Building", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_state_of_building["median_price"].max()*1.1])
    fig.show()

    return df
df = price_corr_state_of_building()

In [ ]:
def price_corr_property_type():
    df = clean_and_load()
    
    # ANOVA: treats postal_code as categorical
    model = ols('Q("price (€)") ~ C(type)', data=df).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)

    """ The ANOVA indicates that property type has a highly significant effect on price (p ≈ 1.06e-83), with a very large F-value showing strong differences between types. 
    However, despite this strong statistical significance, type still explains only part of the total price variability, with most variation coming from other factors."""  

    # Group by location to compute median price and count of listings
    median_price_by_property_type = df.groupby("type", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_property_type["median_price"] = median_price_by_property_type["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_property_type, x="type", y="median_price", color="num_listings", text="median_price", title="Price Variation depending on property type", labels={"type": "Type", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_property_type["median_price"].max()*1.1])
    fig.show()


    return df
df = price_corr_property_type()

In [ ]:
def price_corr_property_subtype():
    df = clean_and_load()
    
    # ANOVA: treats postal_code as categorical
    model = ols('Q("price (€)") ~ C(subtype)', data=df).fit()
    anova_table = sm.stats.anova_lm(model)
    print(anova_table)

    """ The ANOVA shows that property subtype has a highly significant effect on price (p ≈ 0.0), with a very large F-value indicating strong differences between subtypes. 
    Nevertheless, most of the overall price variation remains unexplained, meaning subtype is important but not the primary driver of price."""  

    # Group by location to compute median price and count of listings
    median_price_by_property_subtype = df.groupby("subtype", as_index=False).agg(median_price=("price (€)", "median"), num_listings=("price (€)", "count"))
    median_price_by_property_subtype["median_price"] = median_price_by_property_subtype["median_price"].round(2)

    # bar plot
    fig = px.bar(median_price_by_property_subtype, x="subtype", y="median_price", color="num_listings", text="median_price", title="Price Variation depending on property subtype", labels={"subtype": "Subtype", "median_price": "Median Price (€)", "num_listings": "Number of Listings"})
    fig.update_traces(texttemplate="€%{text:,.0f}", textposition="outside")
    fig.update_yaxes(range=[0, median_price_by_property_subtype["median_price"].max()*1.1])
    fig.show()


    return df
df = price_corr_property_subtype()

Conclusion: 
Top 5 mst important variables influencing the price: 
- Living area (corr. = **0.57**)
- Number of Bedrooms (corr. = **0.43**)
- Terrace area (corr. = **0.43**)
- Swimming pool (corr. = **0.25**)
- Number of facades (corr. = **0.23**)